# RSI STRATEGY

The RSI is a widely used technical indicator and an oscillator that indicates a market is overbought when the RSI value is over 70 and indicates oversold conditions when RSI readings are under 30. Some traders and analysts prefer to use the more extreme readings of 80 and 20. A weakness of the RSI is that sudden, sharp price movements can cause it to spike repeatedly up or down, and, thus, it is prone to giving false signals.

In [1]:
%matplotlib inline  

import pandas as pd
import talib as tl
import numpy as np

In [2]:
def load(file):
    df = pd.read_csv(file, skiprows=1)
    return df.reindex(index=df.index[::-1]).reset_index()

In [3]:
datasets = {
    "BTCUSD": load('Bitstamp_BTCUSD_1h.csv'),
    "ETHUSD": load('Bitstamp_ETHUSD_1h.csv'),
    "LTCUSD": load('Bitstamp_LTCUSD_1h.csv'),
    "XRPUSD": load('Bitstamp_XRPUSD_1h.csv'),
    "ETHBTC": load('Binance_ETHBTC_1h.csv'),
    "LTCBTC": load('Binance_LTCBTC_1h.csv'),
    "XMRBTC": load('Binance_XMRBTC_1h.csv')
}

In [4]:
def benchRSI(df, lowRSI = 30.0, highRSI = 70.):
    data = df.copy()
    data['RSI'] = tl.RSI(data.Close)
    data = data[(data.RSI < 30.) | (data.RSI > 70.)]

    def advice(rsi):
        if rsi<30:
            #buy
            return 'long'
        if rsi>70:
            #sell
            return 'short'
        return np.nan

    data['advice'] = data['RSI'].apply(advice)
    data['prevAdvice'] = data['advice'].shift()
    data = data[data.prevAdvice != data.advice][['Date', 'Close', 'RSI', 'advice']]

    data['enterprice'] = data.Close.shift()

    if data.advice.values[0] == 'short':
        data = data[1:]

    data = data.dropna()
    deals = data[data.advice == 'short'].copy()
    deals['earn'] = (deals.Close - deals.enterprice) / deals.enterprice
    data.drop(['enterprice'], axis=1)
    return deals.earn, data

In [5]:
results = {}
for e in datasets:
    roi, log = benchRSI(datasets[e])
    results[e] = { "roi": roi, "roiAvg": roi.mean(), "log": log }

In [6]:
for e in results:
    print(e)
    print('ntrades', len(results[e]['roi']), ', mean roi',results[e]['roiAvg'])

BTCUSD
ntrades 51 , mean roi 0.0026732290726156184
ETHUSD
ntrades 41 , mean roi -0.035216098055810066
LTCUSD
ntrades 52 , mean roi 0.009105792318809352
XRPUSD
ntrades 50 , mean roi -0.012605270973493005
ETHBTC
ntrades 57 , mean roi -0.02249376560882072
LTCBTC
ntrades 53 , mean roi -0.006085037037699648
XMRBTC
ntrades 39 , mean roi 0.025003177837482283
